# 🇰🇷 한국어 텍스트 교정 모델 파인튜닝 (Google Colab T4)

Google Colab T4 GPU를 사용한 한국어 텍스트 교정 모델 훈련
- 모델: mT5-small
- 기법: LoRA (Parameter-Efficient Fine-tuning)
- 데이터: 구글 드라이브의 CSV 파일들

In [ ]:
# 메모리 최적화 설정 (반드시 최상단에 실행)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:512'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print("⚙️ 메모리 최적화 설정 완료")
print("PYTORCH_CUDA_ALLOC_CONF: expandable_segments:True,max_split_size_mb:512")
print("TOKENIZERS_PARALLELISM: false")
print("CUDA_LAUNCH_BLOCKING: 1")

## 🚀 환경 설정

In [ ]:
# GPU 메모리 관리 및 모니터링 함수들
import gc
import torch

def clear_gpu_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def monitor_gpu_memory():
    """GPU 메모리 사용량 모니터링"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        free = torch.cuda.mem_get_info()[0] / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        used_percent = (allocated / total) * 100
        
        print(f"🔍 GPU 메모리 상태:")
        print(f"  - 할당됨: {allocated:.2f}GB ({used_percent:.1f}%)")
        print(f"  - 예약됨: {reserved:.2f}GB")
        print(f"  - 사용가능: {free:.2f}GB")
        print(f"  - 전체: {total:.2f}GB")
        
        return {
            'allocated_gb': allocated,
            'reserved_gb': reserved,
            'free_gb': free,
            'total_gb': total,
            'used_percent': used_percent
        }
    return None

def get_optimal_batch_size(base_batch_size=8):
    """메모리 상황에 따른 최적 배치 사이즈 계산"""
    memory_info = monitor_gpu_memory()
    if memory_info:
        if memory_info['used_percent'] > 80:
            return max(1, base_batch_size // 4)
        elif memory_info['used_percent'] > 60:
            return max(2, base_batch_size // 2)
        else:
            return base_batch_size
    return base_batch_size

# 초기 메모리 정리
clear_gpu_memory()
print("✅ 메모리 관리 함수들 정의 완료")

In [ ]:
# GPU 확인
!nvidia-smi
print("\nPyTorch CUDA 지원 여부:", torch.cuda.is_available() if 'torch' in globals() else 'torch not imported yet')

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정
import os
os.chdir('/content')
print(f"현재 작업 디렉토리: {os.getcwd()}")

In [ ]:
# 필요한 패키지 설치 (양자화 지원 포함)
!pip install -q transformers datasets peft trl accelerate
!pip install -q evaluate rouge-score sacrebleu scikit-learn
!pip install -q gradio sentencepiece protobuf
!pip install -q bitsandbytes  # 양자화를 위한 패키지
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("✅ 패키지 설치 완료 (양자화 지원 포함)")

In [ ]:
# 메모리 효율성을 위한 환경 변수 설정
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# 필요한 라이브러리 임포트
import torch
import pandas as pd
import numpy as np
import re
import os
import gc
from typing import Dict, List, Any

from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig  # 양자화 지원
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)
from datasets import Dataset
import evaluate

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    # 메모리 사용량 출력
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    free = torch.cuda.mem_get_info()[0] / 1024**3
    print(f"할당된 GPU 메모리: {allocated:.2f}GB")
    print(f"예약된 GPU 메모리: {reserved:.2f}GB")
    print(f"사용 가능한 GPU 메모리: {free:.2f}GB")

## ⚙️ 설정 및 구성

In [ ]:
# 모델 및 훈련 설정 (T4 GPU 양자화 최적화)
CONFIG = {
    "model_name": "google/mt5-small",
    "max_length": 256,
    "batch_size": 4,  # 양자화 환경에서 안전한 크기
    "gradient_accumulation_steps": 8,  # 유효 배치 사이즈 32 유지
    "learning_rate": 1e-4,
    "num_epochs": 3,
    "warmup_steps": 500,
    "logging_steps": 100,
    "save_steps": 500,
    "eval_steps": 500,
    "output_dir": "./korean-text-correction-colab",
}

# LoRA 설정 (양자화 환경 최적화)
LORA_CONFIG = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    "target_modules": ["q", "v", "k", "o", "wi_0", "wi_1", "wo"]
}

# 데이터 파일 경로 (Google Drive)
DATA_FILES = {
    "구어체": "/content/drive/MyDrive/구어체_대화체_16878_sample_난독화결과.csv",
    "뉴스": "/content/drive/MyDrive/뉴스문어체_281932_sample_난독화결과.csv",
    "문화": "/content/drive/MyDrive/문화문어체_25628_sample_난독화결과.csv",
    "전문분야": "/content/drive/MyDrive/전문분야 문어체_306542_sample_난독화결과.csv",
    "조례": "/content/drive/MyDrive/조례문어체_36339_sample_난독화결과.csv",
    "지자체웹사이트": "/content/drive/MyDrive/지자체웹사이트 문어체_28705_sample_난독화결과.csv"
}

# 메모리 상황에 따른 동적 배치 사이즈 조정
optimal_batch_size = get_optimal_batch_size(CONFIG['batch_size'])
if optimal_batch_size != CONFIG['batch_size']:
    print(f"⚠️ 메모리 부족으로 배치 사이즈 조정: {CONFIG['batch_size']} → {optimal_batch_size}")
    CONFIG['batch_size'] = optimal_batch_size
    # Gradient accumulation 조정으로 유효 배치 사이즈 유지
    CONFIG['gradient_accumulation_steps'] = max(1, 32 // optimal_batch_size)

print("✅ 설정 완료 (양자화 최적화)")
print(f"출력 디렉토리: {CONFIG['output_dir']}")
print(f"배치 사이즈: {CONFIG['batch_size']}")
print(f"Gradient accumulation steps: {CONFIG['gradient_accumulation_steps']}")
print(f"유효 배치 사이즈: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")

## 📊 데이터 전처리

In [ ]:
class ColabDataPreprocessor:
    """Google Colab용 데이터 전처리 클래스"""
    
    def __init__(self, max_length: int = 256):
        self.max_length = max_length
        self.text_cleaning_patterns = [
            (r'\s+', ' '),  # 연속된 공백 정리
            (r'^\s+|\s+$', ''),  # 앞뒤 공백 제거
        ]
    
    def clean_text(self, text: str) -> str:
        """텍스트 정리"""
        if pd.isna(text) or text == '':
            return ''
        
        text = str(text)
        for pattern, replacement in self.text_cleaning_patterns:
            text = re.sub(pattern, replacement, text)
        
        return text.strip()
    
    def load_and_process_csv(self, file_path: str, sample_size: int = 5000) -> pd.DataFrame:
        """CSV 파일 로드 및 전처리"""
        print(f"로딩 중: {os.path.basename(file_path)}")
        
        if not os.path.exists(file_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {file_path}")
            return pd.DataFrame()
        
        try:
            # CSV 파일 로드
            df = pd.read_csv(file_path, encoding='utf-8')
            print(f"원본 데이터 크기: {len(df)}")
            print(f"컬럼명: {df.columns.tolist()}")
            
            # 필요한 컬럼 확인 및 매핑
            if 'original' not in df.columns or 'obfuscated' not in df.columns:
                # 한국어 컬럼명이 있는지 확인
                if '원문' in df.columns and '오류문' in df.columns:
                    df = df.rename(columns={'원문': 'original', '오류문': 'obfuscated'})
                    print("✅ 한국어 컬럼명을 영어로 변환했습니다.")
                else:
                    print(f"⚠️ 필요한 컬럼이 없습니다: {df.columns.tolist()}")
                    print("필요한 컬럼: 'original', 'obfuscated' 또는 '원문', '오류문'")
                    return pd.DataFrame()
            
            # 텍스트 정리
            df['original'] = df['original'].apply(self.clean_text)
            df['obfuscated'] = df['obfuscated'].apply(self.clean_text)
            
            # 빈 텍스트 제거
            df = df[(df['original'] != '') & (df['obfuscated'] != '')]
            
            # 길이 제한
            df = df[
                (df['original'].str.len() <= self.max_length) & 
                (df['obfuscated'].str.len() <= self.max_length)
            ]
            
            # 샘플링
            if len(df) > sample_size:
                df = df.sample(n=sample_size, random_state=42)
            
            print(f"전처리 후 데이터 크기: {len(df)}")
            return df
            
        except Exception as e:
            print(f"❌ 파일 로딩 오류: {e}")
            return pd.DataFrame()
    
    def load_all_data(self, data_files: Dict[str, str], sample_per_file: int = 3000) -> pd.DataFrame:
        """모든 데이터 파일 로드 및 통합"""
        all_dfs = []
        
        for name, file_path in data_files.items():
            df = self.load_and_process_csv(file_path, sample_per_file)
            if not df.empty:
                df['source'] = name
                all_dfs.append(df)
        
        if not all_dfs:
            raise ValueError("로드된 데이터가 없습니다.")
        
        combined_df = pd.concat(all_dfs, ignore_index=True)
        combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        print(f"\n📊 통합 데이터셋 정보:")
        print(f"전체 샘플 수: {len(combined_df)}")
        print(f"소스별 분포:")
        print(combined_df['source'].value_counts())
        
        return combined_df

print("✅ 데이터 전처리 클래스 정의 완료")

In [ ]:
# 데이터 로드 및 전처리 (양자화 환경 최적화)
preprocessor = ColabDataPreprocessor(max_length=CONFIG['max_length'])

# 메모리 상태 확인 대이터 로드
print("🔍 데이터 로드 전 메모리 상태:")
memory_info = monitor_gpu_memory()

# 메모리 사용량에 따른 샘플 사이즈 동적 조정
if memory_info and memory_info['used_percent'] > 70:
    sample_per_file = 1500  # 메모리 부족 시 작은 샘플
    print(f"⚠️ 메모리 사용량 높음 ({memory_info['used_percent']:.1f}%), 샘플 사이즈 감소: {sample_per_file}")
else:
    sample_per_file = 2000  # 정상 샘플 사이즈
    print(f"✅ 메모리 상태 양호, 기본 샘플 사이즈 사용: {sample_per_file}")

# 데이터 로드 시작
df = preprocessor.load_all_data(DATA_FILES, sample_per_file=sample_per_file)

# 훈련/검증 분할
train_size = int(0.8 * len(df))
train_df = df[:train_size]
val_df = df[train_size:]

print(f"\n📊 데이터 분할 (양자화 최적화):")
print(f"훈련 데이터: {len(train_df)}")
print(f"검증 데이터: {len(val_df)}")

# 데이터 로드 후 메모리 상태 확인
print(f"\n🔍 데이터 로드 후 메모리 상태:")
monitor_gpu_memory()

# 샘플 데이터 확인
print(f"\n📝 샘플 데이터:")
for i in range(3):
    print(f"오류문 (obfuscated): {train_df.iloc[i]['obfuscated']}")
    print(f"원문 (original): {train_df.iloc[i]['original']}")
    print("-" * 50)

# 메모리 정리
clear_gpu_memory()

## 🤖 모델 설정 및 로드

In [ ]:
# 4-bit 양자화 설정
print("⚙️ 양자화 설정 준비...")

# 4-bit 양자화 설정 (메모리 사용량 ~75% 감소)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

print("📊 양자화 설정:")
print(f"- 4-bit 양자화: {quantization_config.load_in_4bit}")
print(f"- 계산 타입: {quantization_config.bnb_4bit_compute_dtype}")
print(f"- 양자화 타입: {quantization_config.bnb_4bit_quant_type}")
print(f"- 이중 양자화: {quantization_config.bnb_4bit_use_double_quant}")

# 토크나이저 로드
print("\n🔤 토크나이저 로딩...")
tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'], use_fast=False)

# 베이스 모델을 4-bit 양자화로 로드
print("🤖 모델 로딩 (4-bit 양자화)...")
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        CONFIG['model_name'],
        quantization_config=quantization_config,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    print("✅ 양자화된 모델 로드 성공!")
except Exception as e:
    print(f"⚠️ 양자화 로드 실패, 기본 모드로 fallback: {e}")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        CONFIG['model_name'],
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )

# 모델을 kbit 훈련용으로 준비
print("🔧 kbit 훈련용 모델 준비...")
model = prepare_model_for_kbit_training(model)

# LoRA 설정
print("🔧 LoRA 설정 적용...")
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=LORA_CONFIG['r'],
    lora_alpha=LORA_CONFIG['lora_alpha'],
    lora_dropout=LORA_CONFIG['lora_dropout'],
    target_modules=LORA_CONFIG['target_modules']
)

# PEFT 모델 생성
model = get_peft_model(model, lora_config)

# 훈련 가능한 파라미터 확인
model.print_trainable_parameters()

# 메모리 사용량 출력
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"\n📊 모델 로드 후 메모리 사용량:")
    print(f"- 할당된 GPU 메모리: {allocated:.2f}GB")
    print(f"- 예약된 GPU 메모리: {reserved:.2f}GB")

print("\n✅ 양자화 모델 설정 완료!")
print(f"모델 파라미터 수: {model.num_parameters():,}")

# 메모리 정리
clear_gpu_memory()

## 🔤 토크나이제이션

In [ ]:
def tokenize_function(examples):
    """토크나이제이션 함수"""
    # 입력: 오류문(obfuscated), 출력: 원문(original)
    inputs = [f"교정: {text}" for text in examples['obfuscated']]
    targets = examples['original']
    
    # 입력 토크나이제이션
    model_inputs = tokenizer(
        inputs,
        max_length=CONFIG['max_length'],
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    # 타겟 토크나이제이션
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=CONFIG['max_length'],
            truncation=True,
            padding=True,
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 토크나이제이션 적용
print("🔤 토크나이제이션 진행...")
train_tokenized = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

val_tokenized = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("✅ 토크나이제이션 완료")
print(f"훈련 토큰화 데이터: {len(train_tokenized)}")
print(f"검증 토큰화 데이터: {len(val_tokenized)}")

## 🚀 모델 훈련

In [ ]:
# 메트릭 로드
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """평가 메트릭 계산"""
    predictions, labels = eval_pred
    
    # 토큰을 텍스트로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # 라벨에서 -100을 제거
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE 점수 계산
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }

# 데이터 콜레이터
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

print("✅ 훈련 설정 완료")

In [ ]:
# 훈련 인수 설정 (양자화 환경 최적화)
training_args = TrainingArguments(
    output_dir=CONFIG['output_dir'],
    num_train_epochs=CONFIG['num_epochs'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    learning_rate=CONFIG['learning_rate'],
    warmup_steps=CONFIG['warmup_steps'],
    logging_steps=CONFIG['logging_steps'],
    save_steps=CONFIG['save_steps'],
    eval_steps=CONFIG['eval_steps'],
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    report_to=None,  # wandb 등 비활성화
    dataloader_pin_memory=False,  # 메모리 효율성을 위해 비활성화
    fp16=True,  # T4 GPU에서 메모리 효율성을 위해 mixed precision 사용
    gradient_checkpointing=True,  # 메모리 절약을 위한 gradient checkpointing 활성화
    remove_unused_columns=False,
    push_to_hub=False,
    max_grad_norm=1.0,  # Gradient clipping으로 안정성 향상
    save_total_limit=2,  # 디스크 공간 절약
    optim="adamw_torch",  # 메모리 효율적인 optimizer
    dataloader_num_workers=0,  # 메모리 안전성을 위해 비활성화
)

print("📋 훈련 인수 (양자화 최적화):")
print(f"에포크: {CONFIG['num_epochs']}")
print(f"배치 사이즈: {CONFIG['batch_size']}")
print(f"Gradient accumulation: {CONFIG['gradient_accumulation_steps']}")
print(f"유효 배치 사이즈: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")
print(f"학습률: {CONFIG['learning_rate']}")
print(f"출력 디렉토리: {CONFIG['output_dir']}")
print(f"FP16: {training_args.fp16}")
print(f"Gradient Checkpointing: {training_args.gradient_checkpointing}")

In [ ]:
# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ 트레이너 설정 완료")
print("훈련을 시작합니다...")

In [ ]:
# 훈련 시작 (양자화 환경 최적화)
print("🚀 훈련 시작! (양자화 모델)")
print("=" * 50)

# 훈련 전 메모리 상태 확인
print("🔍 훈련 시작 전 메모리 상태:")
monitor_gpu_memory()

try:
    trainer.train()
    print("\n✅ 훈련 완료!")
    
    # 마지막 메모리 상태 확인
    print("\n🔍 훈련 완료 후 메모리 상태:")
    monitor_gpu_memory()
    
    # 최종 모델 저장
    trainer.save_model()
    tokenizer.save_pretrained(CONFIG['output_dir'])
    
    print(f"\n📁 모델 저장 완료: {CONFIG['output_dir']}")
    
except torch.cuda.OutOfMemoryError as e:
    print(f"\n⚠️ GPU 메모리 부족 오류: {e}")
    print("🔧 메모리 정리 후 더 작은 배치 사이즈로 재시도해주세요.")
    
    # 메모리 정리
    clear_gpu_memory()
    
    # 더 작은 배치 사이즈로 재시도
    reduced_batch_size = max(1, CONFIG['batch_size'] // 2)
    print(f"🔄 배치 사이즈를 {CONFIG['batch_size']}에서 {reduced_batch_size}로 감소하여 재시도")
    
except Exception as e:
    print(f"❌ 훈련 중 오류 발생: {e}")
    print("🔍 오류 세부 사항:")
    import traceback
    traceback.print_exc()
    
    # 메모리 정리
    clear_gpu_memory()
    print("🧙 메모리 정리 완료")

finally:
    # 항상 마지막에 메모리 정리
    clear_gpu_memory()
    print("🏁 훈련 세션 종료")

## 📊 모델 평가

In [ ]:
# 최종 평가
print("📊 최종 모델 평가")
eval_results = trainer.evaluate()

print("\n📈 평가 결과:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# 간단한 추론 테스트
def test_correction(text: str) -> str:
    """텍스트 교정 테스트"""
    input_text = f"교정: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

# 테스트 케이스
test_cases = [
    "안녕하셰요",
    "감사햐니다", 
    "잘 뫃겠습니다",
    "괜챠습니까",
    "이거 어떻게 생간하세요"
]

print("\n🧪 텍스트 교정 테스트:")
print("=" * 50)

for test_text in test_cases:
    corrected = test_correction(test_text)
    print(f"원본: {test_text}")
    print(f"교정: {corrected}")
    print("-" * 30)

## 💾 Google Drive에 모델 저장

In [ ]:
# Google Drive에 모델 백업 (양자화 정보 포함)
drive_save_path = "/content/drive/MyDrive/korean-text-correction-model"

print(f"📁 Google Drive에 모델 저장: {drive_save_path}")

# 디렉토리 생성
os.makedirs(drive_save_path, exist_ok=True)

# 양자화 정보 저장
quantization_info = {
    "quantization_type": "4-bit",
    "compute_dtype": "float16",
    "quant_type": "nf4",
    "double_quant": True,
    "lora_config": LORA_CONFIG,
    "training_config": CONFIG
}

# 양자화 정보를 JSON 파일로 저장
import json
with open(f"{drive_save_path}/quantization_info.json", 'w', encoding='utf-8') as f:
    json.dump(quantization_info, f, indent=2, ensure_ascii=False)

try:
    # 모델 파일 복사 (안전하게)
    !cp -r {CONFIG['output_dir']}/* {drive_save_path}/
    
    print("✅ Google Drive 저장 완료")
    print(f"저장된 파일들:")
    !ls -la {drive_save_path}
    
    # 파일 사이즈 체크
    !du -sh {drive_save_path}
    
except Exception as e:
    print(f"⚠️ Google Drive 저장 오류: {e}")
    print("수동으로 파일을 복사해주세요.")

print("\n🎆 양자화 기반 한국어 텍스트 교정 모델 훈련 완료!")
print("🚀 메모리 효율성이 대폭 향상되었습니다.")

## 🧹 메모리 정리

## 🎆 양자화 모델 훈련 요약

### 📊 메모리 최적화 성과
- **4-bit 양자화**: ~75% 메모리 절약
- **LoRA 기법**: 파라미터 효율적 훈련
- **Gradient Checkpointing**: 추가 메모리 절약
- **동적 배치 사이즈**: 메모리 상황에 따른 자동 조정

### 🚀 다음 단계
1. **사용법**: `colab_inference.ipynb`로 모델 테스트
2. **모델 위치**: `/content/drive/MyDrive/korean-text-correction-model`
3. **양자화 정보**: `quantization_info.json` 파일 참조

### ⚠️ 메모리 관리 팁
- 훈련 중 OOM 오류 시 배치 사이즈 감소
- `clear_gpu_memory()` 함수로 정기적 메모리 정리
- `monitor_gpu_memory()`로 실시간 메모리 모니터링

In [ ]:
# 전체 메모리 정리 및 운력 모니터링
print("🗑️ 전체 메모리 정리 시작...")

# 마지막 메모리 상태 확인
print("\n🔍 정리 전 메모리 상태:")
final_memory_before = monitor_gpu_memory()

# 모델과 트레이너 객체 삭제
if 'model' in globals():
    del model
if 'trainer' in globals():
    del trainer

# 전체 메모리 정리
clear_gpu_memory()

print("\n🔍 정리 후 메모리 상태:")
final_memory_after = monitor_gpu_memory()

# 메모리 절약 효과 계산
if final_memory_before and final_memory_after:
    memory_freed = final_memory_before['allocated_gb'] - final_memory_after['allocated_gb']
    print(f"\n💾 메모리 절약 효과: {memory_freed:.2f}GB 해제")

print("\n✅ 메모리 정리 완료")
print("\n🎉 훈련이 성공적으로 완료되었습니다!")
print(f"📁 훈련된 모델은 다음 위치에 저장되었습니다:")
print(f"- Colab: {CONFIG['output_dir']}")
print(f"- Google Drive: /content/drive/MyDrive/korean-text-correction-model")
print("\n🚀 이제 colab_inference.ipynb를 사용하여 모델을 테스트해보세요!")